# RFM-analysis

Connect the module for working with data

In [1]:
import pandas as pd

orders = pd.read_csv('data.csv',sep=',')

Output the data tables.

In [2]:
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franz�sisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


Проверим есть ли в данных нулевые значения

In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_date   5009 non-null   object
 1   order_id     5009 non-null   object
 2   customer     5009 non-null   object
 3   grand_total  5009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 156.7+ KB


In [4]:
orders.isna().sum()

order_date     0
order_id       0
customer       0
grand_total    0
dtype: int64

Приведём данные к тому формату, с которым можно работать.

In [5]:
orders['order_date'] = pd.to_datetime(orders['order_date'])
orders['customer'] = orders['customer'].apply(str)

In [6]:
orders.head()

,order_date,order_id,customer,grand_total
0,2011-09-07,CA-2011-100006,Dennis Kane,378
1,2011-07-08,CA-2011-100090,Ed Braxton,699
2,2011-03-14,CA-2011-100293,Neil Franz�sisch,91
3,2011-01-29,CA-2011-100328,Jasper Cacioppo,4
4,2011-04-08,CA-2011-100363,Jim Mitchum,21


In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_date   5009 non-null   datetime64[ns]
 1   order_id     5009 non-null   object        
 2   customer     5009 non-null   object        
 3   grand_total  5009 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 156.7+ KB


Найдём самую позднюю покупку.

In [8]:
orders.max()

order_date     2014-12-31 00:00:00
order_id            US-2014-169551
customer        Zuschuss Donatelli
grand_total                  23661
dtype: object

Создадим перемнную по отношению к которой, мы будем в последующем считать

In [9]:
import datetime as dt 
NOW = dt.datetime(2014, 12,31)

Нам нужно найти все данные по каждому клиенту и их преобразовать особым образом, чтобы подсчитать три показателя RFM-анализа.

In [10]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency # Количество дней с последнего заказа
                                        'order_id': lambda x: len(x),      # Frequency # Количество заказов
                                        'grand_total': lambda x: x.sum()}) # Monetary Value # Общая сумма по всем заказам

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

In [11]:
rfmTable


,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249
...,...,...,...
Xylona Preis,43,11,2375
Yana Sorensen,9,8,6721
Yoseph Carroll,4,5,5455


Теперь мы можем сделать разбивку по квантилям. Возьмем четыре параметра: 0,25 0,5 0,75 и 1. В Python имеется встроенная функция, которая позволит выполнить эту задачу:

In [12]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [13]:
quantiles

,recency,frequency,monetary_value
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


Чтобы нам было удобнее работать дальше с этими данными, их необходимо преобразовать в словарь.

Теперь мы можем приступить к формированию таблицы сегментации:

In [14]:
rfmSegmentation = rfmTable 

In [15]:
rfmSegmentation

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249
...,...,...,...
Xylona Preis,43,11,2375
Yana Sorensen,9,8,6721
Yoseph Carroll,4,5,5455


Создадим вспомогательную таблицу, в которую и будем вносить все дальнейшие изменения.

Здесь важно учесть, что показатель давности покупки будет лучше, если он меньше, поэтому создадим с вами две отдельные функции. Одна функция будет работать с показателем R, другая — с F и M.

In [16]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

Далее применим эти функции для нахождения значений по показателям у каждого покупателя:

In [17]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

Теперь можем получить итоговое значение:

In [18]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [19]:
rfmSegmentation

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,415,3,887,4,4,4,444
Aaron Hawkins,12,7,1744,1,2,3,123
Aaron Smayling,88,7,3050,3,2,2,322
Adam Bellavance,54,8,7756,2,2,1,221
Adam Hart,34,10,3249,2,1,2,212
...,...,...,...,...,...,...,...
Xylona Preis,43,11,2375,2,1,2,212
Yana Sorensen,9,8,6721,1,2,1,121
Yoseph Carroll,4,5,5455,1,4,1,141


In [21]:
orders_new = pd.read_csv('RFM201810.csv',sep=',')

In [22]:
orders_new.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2018-10-01 00:00:00.000,1716.00
1,C0011810010017,13233933,2018-10-01 00:00:00.000,1489.74
2,C0011810010020,99057968,2018-10-01 00:00:00.000,151.47
3,C0011810010021,80007276,2018-10-01 00:00:00.000,146.72
4,C0011810010024,13164076,2018-10-01 00:00:00.000,104.00


In [23]:
data = pd.read_csv('RFM201810.csv', low_memory = False)

In [24]:
data.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2018-10-01 00:00:00.000,1716.00
1,C0011810010017,13233933,2018-10-01 00:00:00.000,1489.74
2,C0011810010020,99057968,2018-10-01 00:00:00.000,151.47
3,C0011810010021,80007276,2018-10-01 00:00:00.000,146.72
4,C0011810010024,13164076,2018-10-01 00:00:00.000,104.00


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     332730 non-null  object 
 1   CustomerCode  332730 non-null  object 
 2   InvoiceDate   332730 non-null  object 
 3   Amount        332730 non-null  float64
dtypes: float64(1), object(3)
memory usage: 10.2+ MB


In [29]:
orders.head()

,order_date,order_id,customer,grand_total
0,2011-09-07,CA-2011-100006,Dennis Kane,378
1,2011-07-08,CA-2011-100090,Ed Braxton,699
2,2011-03-14,CA-2011-100293,Neil Franz�sisch,91
3,2011-01-29,CA-2011-100328,Jasper Cacioppo,4
4,2011-04-08,CA-2011-100363,Jim Mitchum,21


In [26]:
data.isna().sum()

InvoiceNo       0
CustomerCode    0
InvoiceDate     0
Amount          0
dtype: int64

In [27]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['CustomerCode'] = data['CustomerCode'].apply(str)

In [28]:
data.max()

InvoiceNo            S0081810310466
CustomerCode               99099972
InvoiceDate     2018-10-31 00:00:00
Amount                       131874
dtype: object

In [ ]:
NOW = dt.datetime(2014, 12,31)